### DMD Ingredient->Product Lookup

Get a list of all VMPs (Virtual Medicinal Products) and AMPs (Actual Medicinal Products) from a given list of ingredients (VTMs = Virtual Therapeutic Moieties)

In [1]:
# set name of codelist for exporting file
codelist_name="covid_meds"


# import or paste list of vtms (Virtual Therapeutic Moieties / Ingredients) by name
names = ['Casirivimab', 'Imdevimab', 'Remdesivir', 'Tocilizumab', 'Sarilumab']




from ebmdatalab import bq
import os
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)



## Select products from DMD containing chosen ingredients

In [2]:
# convert list to tuple for use in SQL query
names_tuple = tuple(names)
if len(names_tuple)==1:
    # remove comma if only one item
    names_tuple = str(names_tuple).replace(",","")

sql = f'''
SELECT "vmp" AS type, vmp.id, bnf_code, vmp.nm, ing.nm AS ingredient
FROM dmd.vmp
INNER JOIN dmd.vpi AS vpi ON vmp.id=vpi.vmp 
INNER JOIN dmd.ing as ing ON ing.id = vpi.ing AND ing.nm IN {names_tuple}

UNION ALL

SELECT "amp" AS type, amp.id, bnf_code, descr as nm, ing.nm as ingredient
FROM dmd.amp
INNER JOIN dmd.vpi AS vpi ON amp.vmp=vpi.vmp 
INNER JOIN dmd.ing as ing ON ing.id = vpi.ing AND ing.nm IN {names_tuple}

ORDER BY type, nm  '''

meds = bq.cached_read(sql, csv_path=os.path.join('..','data',f'meds_{codelist_name}.csv'))

meds.count()

C:\Users\hcurtis\Anaconda3\envs\antibiotics-rct\lib\site-packages\pandas_gbq\gbq.py:547: UserWarning: A progress bar was requested, but there was an error loading the tqdm library. Please install tqdm to use the progress bar functionality.
  df = rows_iter.to_dataframe(


type          36
id            36
bnf_code      18
nm            36
ingredient    36
dtype: int64

#### Check if any ingredients were not found

In [3]:
ings = list(meds.ingredient.drop_duplicates())
names.sort()
ings.sort()
print("Selected ingredients:", names)
print("Ingredients found:", ings)

missing_ings = [i for i in names if i not in ings]
print("Ingredients NOT found:", missing_ings)

Selected ingredients: ['Casirivimab', 'Imdevimab', 'Remdesivir', 'Sarilumab', 'Tocilizumab']
Ingredients found: ['Casirivimab', 'Imdevimab', 'Remdesivir', 'Sarilumab', 'Tocilizumab']
Ingredients NOT found: []
